#  Processors and Utils

Description of the main tools and utilities that one needs to prepare the data for a `WideDeep` model constructor. 

#### The `preprocessing`  module

There are 4 preprocessors, corresponding to 4 main components of the `WideDeep` model. These are

* `WidePreprocessor`
* `TabPreprocessor`
* `TextPreprocessor`
* `ImagePreprocessor` 

Behind the scenes, these preprocessors use a series of helper funcions and classes that are in the `utils` module. If you were interested please go and have a look to the documentation

##  1. WidePreprocessor

The `wide` component of the model is a linear model that in principle, could be implemented as a linear layer receiving the result of on one-hot encoding categorical columns. However, this is not memory efficient. Therefore, we implement a liner layer as an Embedding layer plus a bias. I will explain in a bit more detail later. 

With that in mind, `WidePreprocessor` simply encodes the categories numerically so that they are the indexes of the lookup table that is an Embedding layer.

For example

In [1]:
import numpy as np
import pandas as pd
import pytorch_widedeep as wd

from pytorch_widedeep.datasets import load_adult
from pytorch_widedeep.preprocessing import WidePreprocessor

/Users/javierrodriguezzaurin/.pyenv/versions/3.8.12/envs/wd38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = load_adult(as_frame=True)
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
wide_cols = [
    "education",
    "relationship",
    "workclass",
    "occupation",
    "native-country",
    "gender",
]
crossed_cols = [("education", "occupation"), ("native-country", "occupation")]

In [4]:
wide_preprocessor = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)
X_wide = wide_preprocessor.fit_transform(df)
# From here on, any new observation can be prepared by simply running `.transform`
# new_X_wide = wide_preprocessor.transform(new_df)

In [5]:
X_wide

array([[  1,  17,  23, ...,  89,  91, 316],
       [  2,  18,  23, ...,  89,  92, 317],
       [  3,  18,  24, ...,  89,  93, 318],
       ...,
       [  2,  20,  23, ...,  90, 103, 323],
       [  2,  17,  23, ...,  89, 103, 323],
       [  2,  21,  29, ...,  90, 115, 324]])

Note that the label encoding starts from `1`. This is because it is convenient to leave `0` for padding, i.e. unknown categories. Let's take from example the first entry

In [6]:
X_wide[0]

array([  1,  17,  23,  32,  47,  89,  91, 316])

In [7]:
wide_preprocessor.inverse_transform(X_wide[:1])

,education,relationship,workclass,occupation,native-country,gender,education_occupation,native-country_occupation
0,11th,Own-child,Private,Machine-op-inspct,United-States,Male,11th-Machine-op-inspct,United-States-Machine-op-inspct


As we can see, `wide_preprocessor` numerically encodes the `wide_cols` and the `crossed_cols`, which can be recovered using the method `inverse_transform`.

##  2. TabPreprocessor

Label encodes the categorical columns and normalises the numerical ones (unless otherwised specified).

In [8]:
from pytorch_widedeep.preprocessing import TabPreprocessor

In [9]:
# cat_embed_cols = [(column_name, embed_dim), ...]
cat_embed_cols = [
    ("education", 10),
    ("relationship", 8),
    ("workclass", 10),
    ("occupation", 10),
    ("native-country", 10),
]
continuous_cols = ["age", "hours-per-week"]

In [10]:
tab_preprocessor = TabPreprocessor(
    cat_embed_cols=cat_embed_cols, continuous_cols=continuous_cols
)
X_tab = tab_preprocessor.fit_transform(df)
# From here on, any new observation can be prepared by simply running `.transform`
# new_X_deep = deep_preprocessor.transform(new_df)

In [11]:
X_tab

array([[ 1.        ,  1.        ,  1.        , ...,  1.        ,
        -0.99512893, -0.03408696],
       [ 2.        ,  2.        ,  1.        , ...,  1.        ,
        -0.04694151,  0.77292975],
       [ 3.        ,  2.        ,  2.        , ...,  1.        ,
        -0.77631645, -0.03408696],
       ...,
       [ 2.        ,  4.        ,  1.        , ...,  1.        ,
         1.41180837, -0.03408696],
       [ 2.        ,  1.        ,  1.        , ...,  1.        ,
        -1.21394141, -1.64812038],
       [ 2.        ,  5.        ,  7.        , ...,  1.        ,
         0.97418341, -0.03408696]])

Note that the label encoding starts from `1`. This is because it is convenient to leave `0` for padding, i.e. unknown categories. Let's take from example the first entry

In [12]:
X_tab[0]

array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
       -0.99512893, -0.03408696])

In [13]:
tab_preprocessor.inverse_transform(X_tab[:1])

,education,relationship,workclass,occupation,native-country,age,hours-per-week
0,11th,Own-child,Private,Machine-op-inspct,United-States,25.0,40.0


##  3. TextPreprocessor

This preprocessor returns the tokenised, padded sequences that will be directly fed to the stack of LSTMs.

In [14]:
from pytorch_widedeep.preprocessing import TextPreprocessor

In [15]:
# The airbnb dataset, which you could get from here:
# http://insideairbnb.com/get-the-data.html, is too big to be included in
# our datasets module (when including images). Therefore, go there,
# download it, and use the download_images.py script to get the images
# and the airbnb_data_processing.py to process the data. We'll find
# better datasets in the future ;). Note that here we are only using a
# small sample to illustrate the use, so PLEASE ignore the results, just
# focus on usage
df = pd.read_csv("../tmp_data/airbnb/airbnb_sample.csv")

In [16]:
texts = df.description.tolist()
texts[:2]

["My bright double bedroom with a large window has a relaxed feeling! It comfortably fits one or two and is centrally located just two blocks from Finsbury Park. Enjoy great restaurants in the area and easy access to easy transport tubes, trains and buses. Babies and children of all ages are welcome. Hello Everyone, I'm offering my lovely double bedroom in Finsbury Park area (zone 2) for let in a shared apartment.  You will share the apartment with me and it is fully furnished with a self catering kitchen. Two people can easily sleep well as the room has a queen size bed. I also have a travel cot for a baby for guest with small children.  I will require a deposit up front as a security gesture on both our parts and will be given back to you when you return the keys.  I trust anyone who will be responding to this add would treat my home with care and respect .  Best Wishes  Alina Guest will have access to the self catering kitchen and bathroom. There is the flat is equipped wifi interne

In [17]:
text_preprocessor = TextPreprocessor(text_col="description")
X_text = text_preprocessor.fit_transform(df)
# From here on, any new observation can be prepared by simply running `.transform`
# new_X_text = text_preprocessor.transform(new_df)

The vocabulary contains 2192 tokens


In [18]:
print(X_text[0])

[  29   48   37  367  818   17  910   17  177   15  122  349   53  879
 1174  126  393   40  911    0   23  228   71  819    9   53   55 1380
  225   11   18  308   18 1564   10  755    0  942  239   53   55    0
   11   36 1013  277 1974   70   62   15 1475    9  943    5  251    5
    0    5    0    5  177   53   37   75   11   10  294  726   32    9
   42    5   25   12   10   22   12  136  100  145]


## 4. ImagePreprocessor

`ImagePreprocessor` simply resizes the images, being aware of the aspect ratio.  

In [19]:
from pytorch_widedeep.preprocessing import ImagePreprocessor

In [20]:
image_preprocessor = wd.preprocessing.ImagePreprocessor(
    img_col="id", img_path="../tmp_data/airbnb/property_picture/"
)
X_images = image_preprocessor.fit_transform(df)
# From here on, any new observation can be prepared by simply running `.transform`
# new_X_images = image_preprocessor.transform(new_df)

Reading Images from ../tmp_data/airbnb/property_picture/
Resizing


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1001/1001 [00:01<00:00, 601.56it/s]


Computing normalisation metrics


In [21]:
X_images[0].shape

(224, 224, 3)